In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# função pra se certificar de que uma coluna (indicada a seguir) terá campo tipo float.
# Os números das casas são na realidade inteiros, mas aqui optou-se por trabalhar com tipo float porque esse tipo de campo
# comporta valores nulos. Usa-se valores nulos aqui porque não existe numeração 0 em ruas - o tipo integer obriga a usar 0
# ao invés de nulo - inviabilizando a busca pelo menor número não sendo 0.

def conv(val):
    if not val:
        return None 
    try:
        return np.float64(val)
    except:        
        return np.float64()

# lê arquivo .csv, separado por ';' e já seleciona colunas que serão usadas (deixando de fora colunas inúteis para esse exercício).
# Chama a função acima para converter o tipo de campo das colunas que serão tipo float.
    
StreetMapCSV = r'D:\Henrique\VectorStreets_BR.csv'
df = pd.read_csv(StreetMapCSV, sep = ';', usecols=['StreetName',
       'LEFT_START_ADDRESS', 'LEFT_END_ADDRESS', 'RIGHT_START_ADDRESS',
       'RIGHT_END_ADDRESS', 'LEFT_POSTAL', 'RIGHT_POSTAL', 'REGION', 'PROVINCE', 'CITY'], 
                 converters={'LEFT_START_ADDRESS':conv, 'LEFT_END_ADDRESS':conv, 'RIGHT_START_ADDRESS':conv,
       'RIGHT_END_ADDRESS':conv})
df.head(10)

In [ ]:
# Imprime os nomes das colunas para facilitar renomeação dos campos.

print(df.columns)

In [ ]:
# Informações das colunas para conferência dos tipos de campo.

df.info()

In [ ]:
# renomeia colunas e imprime. "inplace-True" para que o 'df' original receba o comando "rename".

df.rename(columns={'StreetName': 'Logradouro', 'LEFT_START_ADDRESS': 'Num_Inicio_ESQ', 'LEFT_END_ADDRESS': 'Num_Final_ESQ',
       'RIGHT_START_ADDRESS': 'Num_Inicio_DIR', 'RIGHT_END_ADDRESS': 'Num_Final_DIR', 'LEFT_POSTAL': 'CEP_ESQ',
       'RIGHT_POSTAL': 'CEP_DIR', 'REGION': 'Regiao_BR', 'PROVINCE': 'Estado',
       'CITY': 'Cidade'}, inplace=True)
print(df.columns)

In [ ]:
# separação do DataFrame original em dois para reestruturação da planilha.
# Aqui cria-se o DF para os endereços da esquerda do vetor. Usa-se o método "drop" para
# apagar os campos "da direita". "axis=1" para que esse comando aconteça nas colunas.

df_left = df.drop(['Num_Inicio_DIR', 'Num_Final_DIR', 'CEP_DIR'], axis=1)
df_left.head()

In [ ]:
# Criação do DF para os endereços à direita usando-se "drop".

df_right = df.drop(['Num_Inicio_ESQ', 'Num_Final_ESQ', 'CEP_ESQ'], axis=1)
df_right.head()

In [ ]:
# cria variáveis para renomaear novamente as colunas dos dois novos DFs para deixá-las iguais preparando para a concatenação.
# Foi necessário manter a diferença dos nomes das colunas durante etapas anteriores para ser possível separar os novos DFs.
# Concatenação entre os dois novos DFs.

colunas_left_rename = {'Num_Inicio_ESQ': 'Num_Inicio', 'Num_Final_ESQ': 'Num_Final',
       'CEP_ESQ': 'CEP'}
colunas_right_rename = {'Num_Inicio_DIR': 'Num_Inicio', 'Num_Final_DIR': 'Num_Final',
       'CEP_DIR': 'CEP'}
df_total = pd.concat([df_left.rename(columns=colunas_left_rename), df_right.rename(columns=colunas_right_rename)])
df_total.info()

In [ ]:
# conferência da estrutura do novo DF.

df_total.head()

In [ ]:
# Processo de limpeza (descarte) das linhas que tem valores nulos ('NaN').
# Essa função não será executada apenas nas colunas numéricas para que possa haver linhas com CEP, logradouro e cidade,
# mesmo sem numeração de rua.

df_total_clean = df_total.dropna(subset=['Logradouro', 'CEP'])
df_total_clean.head(20)

In [ ]:
# conferindo a eficácia da limpeza das colunas 'CEP' e 'logradouro', através de função de soma dos valores nulos por coluna.

df_total_clean.isnull().sum()

In [ ]:
df_total_clean.info()

In [ ]:
# Função agrupar para realizar a seleção dos números maiores e menores de cada uma das colunas numéricas.
# Criação de novo dataframe com as colunas CEP e logradouro agrupadas com os valores mínimo e máximo da numeração da rua
# inicial e final, mais as respectivas colunas 'Regiao_BR', 'Estado' e 'Cidade'.
# função reset do índice do novo dataframe.

df_groupby = df_total_clean.groupby(['CEP', 'Logradouro'])

min_NumIni = df_groupby['Num_Inicio'].min()
max_NumIni = df_groupby['Num_Inicio'].max()
min_NumFin = df_groupby['Num_Final'].min()
max_NumFin = df_groupby['Num_Final'].max()
Regiao_BR = df_groupby['Regiao_BR'].first()
Estado = df_groupby['Estado'].first()
Cidade = df_groupby['Cidade'].first()

CEP_Log_Nums = pd.DataFrame({'min_NumIni': min_NumIni, 'max_NumIni': max_NumIni, 'min_NumFin': min_NumFin, 
                             'max_NumFin': max_NumFin, 'Regiao_BR': Regiao_BR, 'Estado': Estado, 'Cidade': Cidade})

print(CEP_Log_Nums.reset_index())

In [ ]:
# gera novas colunas no dataframe.

CEP_Log_Nums['Menor_Num_Lograd'] = 0
CEP_Log_Nums['Maior_Num_Lograd'] = 0

In [ ]:
CEP_Log_Nums.info()

In [ ]:
# seleciona o maior e o menor número da rua dentre os quatro campos de numeração (máximo e mínimo final e inicial). 

CEP_Log_Nums['Menor_Num_Lograd'] = CEP_Log_Nums.loc[:, ['min_NumIni', 'max_NumIni', 'min_NumFin', 'max_NumFin']].min(1)
CEP_Log_Nums['Maior_Num_Lograd'] = CEP_Log_Nums.loc[:, ['min_NumIni', 'max_NumIni', 'min_NumFin', 'max_NumFin']].max(1)

In [ ]:
# Cria novo DataFrame dispensando as quatro colunas, agora desnecessárias.

CEP_Log_Nums_columnFinal = CEP_Log_Nums.drop(['min_NumIni', 'max_NumIni', 'min_NumFin', 'max_NumFin'], axis=1)

In [ ]:
# valores nulos são substituidos por 0 nas duas colunas numéricas.

CEP_Log_Nums_columnFinal['Menor_Num_Lograd'].fillna(0, inplace = True)
CEP_Log_Nums_columnFinal['Maior_Num_Lograd'].fillna(0, inplace = True)

In [ ]:
CEP_Log_Nums_columnFinal.info()

In [ ]:
# converte números as colunas dos números de float para integer. Assim a numeração da rua fica sem vírgula, como na realidade.

CEP_Log_Nums_columnFinal[['Menor_Num_Lograd', 'Maior_Num_Lograd']] = CEP_Log_Nums_columnFinal[['Menor_Num_Lograd', 'Maior_Num_Lograd']].astype(int)

In [ ]:
CEP_Log_Nums_columnFinal.info()

In [ ]:
CEP_Log_Nums_columnFinal.isnull().any()

In [ ]:
CEP_Log_Nums_columnFinal

In [ ]:
# Exporta um csv do DataFrame final, separado por ';' e com acentuação em português.

CEP_Log_Nums_columnFinal.to_csv(r'D:\Henrique\Postal_Address_BR.csv',
                                encoding='iso-8859-1',sep=';')
print('Processo Concluído')